## Importations :

In [1]:
# Data loading and manipulation
import pandas as pd
import numpy as np
# K-Means clustering algorithm
from sklearn.cluster import KMeans
# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
# Places the plots in the Jupyter Notebook
%matplotlib inline
# PCA dimension reduction
from sklearn.decomposition import PCA
from sklearn import preprocessing
# Keep everything visible
pd.set_option('display.max_columns', 80)
pd.set_option('expand_frame_repr', True)

import bandits as ba

# Fonctions :

### Chargement données : 

Fonction parcourant le répertoire "npb_fast", et récupérant les startTimes des expériences dont la fréquence d'exécution correspod à l'entrée de la fonction.



Entrée :
- freq : Entier : fréquence d'exécution de référence



Sortie :
- df : dataframe contenant les données d'identification des applications exécutées à la fréquence de référence.

In [2]:
def load_data(freq):
    list_freq = [1200000,1300000,1400000,1500000,1600000,1700000,1800000,1900000,2000000,2100000,2200000,2300000,2400000]

    if freq not in list_freq :
        raise ValueError('Incorrect frequency')

    path ='/home/agary/Documents/Data/npb_fast/02_remove_wm/'
    f1 = 'npb_graoully-'
    f2 = ['1','1','1','1','1','2','3','6','10','10','11','11','12','12','13','13','13','13','14','15','16']
    f3 = '.nancy.grid5000.fr_'
    f4 = ['1637601437','1637609938','1637875436','1637899219','1637920836','1637601340','1637525287','1637920832','1637525761','1637601438','1637609677','1637875436','1637898864','1637899935','1636306023','1637525287','1637899037','1637899639','1637920837','1637609637','1637875436']
    f5 = '_cleaned.csv'


    filename = f1+f2[0]+f3+f4[0]+f5
    df_t = pd.read_csv(path + filename, sep=' ')
    df = df_t.loc[df_t['fmax']==freq]

    for i in range(1,len(f2)):
        filename = f1+f2[i]+f3+f4[i]+f5
        temp = pd.read_csv(path + filename, sep=' ')
        temp = temp.loc[temp['fmax']==freq]

        df = pd.concat([df,temp])
        
    return df

### Clustering :

Fonction récupérant les données détaillées des expériences en allant les chercher dans les dossiers correspondant, puis créé des clusters avec une méthode kmeans sur ces données. Les clusters sont créés sur les données du début de l'exécution des applications. Les 4 premières secondes sont enlevées car c'est sur cette période là que l'application démare et charge toutes les données dont elle a besoin, ce temps n'est donc pas vraiment représentatif du comportement global de l'application. Les clusters sont créés en utilisant la valeur médiane pour chaque covariable.



Entrée : 
- df : dataframe créé à l'aide de la fonction "load_data", comprenant les données d'idenitifation de chaque application.
- t : entier : période de temps sur laquelle les données sont récupérées.
- c : nombre de clusters à créer



Sortie :
- dataframe0 : dataframe contenant les données détaillées de chaque application, avec en plus l'index du cluster auquel elle a été associée.
- Kmeans0 : clustering effecuté avec algorithme Kmeans.

In [3]:
def clustering(df,t,c):
    
    if t <= 4 :
        raise ValueError("time t must be >= 4")
        
    path ='/home/agary/Documents/Data/npb_fast/02_remove_wm/'    
    m,n = df.shape
    machines = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16']
    dataframe0 = pd.DataFrame()
    list_server = []
    list_app = []
    list_duration = []
    list_basename = []

    for i in range(m):

        basename = df.iat[i,9]
        path_temp = path + basename + '_mojitos/' # creation du chemin jusqu'au fichier correspondant
        key = df.iat[i,4]
        app = df.iat[i,1]

        for j in machines :

            name = 'graoully-' + j + '.nancy.grid5000.fr_' + app + '_' + str(key)
            filename = path_temp + name

            # test si fichier existe : si oui : manip :

            if os.path.exists(filename) :
                dt = pd.read_csv(filename,sep=' ')
                time = dt.iat[0,0]
                end = dt.iat[-1,0]
                dt_0 = dt[dt['#timestamp']<=time+t]
                dt_partial = dt[dt['#timestamp']>=time+4]
                dt_median = dt_partial.median()

                dataframe0 = dataframe0.append(dt_median,ignore_index=True)
                list_server.append(j)
                list_basename.append(basename)
                list_app.append(app)
                list_duration.append(end-time)

    dataframe0['server'] = list_server
    dataframe0['basename'] = list_basename
    dataframe0['app'] = list_app
    dataframe0['duration'] = list_duration
    dataframe0 = dataframe0.drop(['#timestamp'],axis=1)
    
    dt_final0 = dataframe0.drop(['server','app','basename'],axis=1)
    #print(dt_final0.isnull().sum().sum())
    kmeans0 = KMeans(n_clusters=c)
    kmeans0.fit(dt_final0);

    dataframe0['kmeans_labels'] = kmeans0.labels_
    
    return dataframe0,kmeans0

### Bandits :

#### Calcul_freq :

cf notebook "test raw_data" : la seule différence est ici que l'on a la liste des clusters à la place de la liste des noms des applications.

In [4]:
def calcul_freq(data,res_linucb):

    liste_clusters = [0,1,2,3,4]
    frequences = [1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3,2.4]
    
    temps_execution = []
    consommations = []
    best_consos = []
    
    best_freq = [] # On stocke la frequences optimale
    gains = [] # On stocke les gains moyens
    
    for k in range(len(liste_clusters)) :
        clust = liste_clusters[k]
        
        # Calcul des temps moyens :

        indices_clusters = np.where(data['kmeans_labels'] == clust)[0] #On récupère les indices associés à l'application
        n = data.shape[0]
        temps = np.zeros(13)

        for i in range(n):
            if i in indices_clusters :
                for j in range(13):
                    
                    temps[j] += data.iat[i,j+1]

        temps /= len(indices_clusters)
        temps_execution.append(temps)
        
        # Calcul de la consommation prévue
        
        consos = []
        for i in range(13):
            c = res_linucb.iat[i,k]
            d = res_linucb.iat[i,-1]
        

            energie = abs(c + temps[i]*d)
            consos.append(energie)

        consommations.append(consos)
        indices_min = np.argmin(consos)
        best_consos.append(min(consos))

        best_freq.append(frequences[indices_min])
        
        # Construction du tableau recaptulatif :
        
        consos.remove(best_consos[-1])
        moy = np.mean(consos)

        if moy == 0 :
            gains.append('NA')
        else :
            gains.append(moy - best_consos[-1])
        
    a = np.array([best_freq, best_consos, gains])
    a = np.transpose(a)
    recap = pd.DataFrame(data=a, index=liste_clusters, columns=['Frequence optimale', 'Consommation optimale', 'Gain moyen'])
    return [recap, temps_execution, consommations]

#### Bandits :

Fonction récupérant le dataframe0 créé suite au clustering, récupérant les données de durée et de consommation énergétique pour chaque expérience et créé 2 dataframes : un pour le contexte (index des clusters binarisés et durée) et un pour les récompenses (consommations énergétiques passées en négatif). Enfin, l'algorithme LINUCB est appliqué sur les deux dataframes ainsi créés pour déterminer la fréquence optimale pour chaque cluster (via la fonction calcul_freq).



Entrée :
- dataframe0 : dataframe contenant les données détaillées de chaque expérience, ainsi que le cluster correspondant à chaque fois.



Sortie :
- recap : dataframe contenant la fréquence optimale pour chaque cluster, avec le gain moyen en terme de consommation par rapport aux autres choix de fréquences.

In [5]:
def bandits(dataframe0):

    rewards = pd.DataFrame(dataframe0[['app','basename']])
    m,n = rewards.shape
    path = '/home/agary/Documents/Data/npb_fast/02_remove_wm/'
    energies = np.zeros((13,m))
    durations = np.zeros((13,m))

    for i in range(m):

        filename = rewards.iat[i,1]+'_cleaned.csv'
        data = pd.read_csv(path+filename,sep=' ')    
        data = data.loc[data['fullname']==rewards.iat[i,0]]

        if data.shape[0] > 13 : # lorsqu'une même application soit testée plusieurs fois dans un même fichier
            s = data.shape[0] // 13
            en = list(data['energy'])
            du = list(data['time'])
            for j in range(s):
                en_temp = en[int(j*13):int((j+1)*13)]
                du_temp = du[int(j*13):int((j+1)*13)]
                energies[:,i] = en_temp
                durations[:,i] = du_temp

        else :
            energies[:,i] = list(data['energy'])
            durations[:,i] = list(data['time'])

    energies = np.transpose(energies)
    durations = np.transpose(durations)

    rewards = pd.concat([rewards,pd.DataFrame(energies)],axis=1)
    list_rewards = ['ref_energy','ref_energy.1','ref_energy.2','ref_energy.3','ref_energy.4','ref_energy.5','ref_energy.6','ref_energy.7','ref_energy.8','ref_energy.9','ref_energy.10','ref_energy.11','ref_energy.12']
    rewards = rewards.drop(['app','basename'],axis=1)
    rewards.columns = list_rewards

    context = pd.get_dummies(dataframe0['kmeans_labels'])
    context['duration'] = dataframe0['duration']
    list_durations = ['duration','duration.1','duration.2','duration.3','duration.4','duration.5','duration.6','duration.7','duration.8','duration.9','duration.10','duration.11','duration.12']
    durations = pd.DataFrame(durations,columns=list_durations)

    dt = dataframe0['kmeans_labels'].copy()
    dt = pd.concat([dt,durations,rewards],axis=1)

    rewards = (-1)*rewards
    alloc_linucb = ba.LINUCB(context,rewards)
    
    recap,temps_execution,consommations = calcul_freq(dt,alloc_linucb[1])
    
    return recap

### Clustering & Bandits :

Fonction regroupant celles vues ci-dessus.

In [6]:
def clustering_bandits(freq,t,c):
    
    df = load_data(freq)
    
    dataframe0,Kmeans0 = clustering(df,t,c)
    
    recap = bandits(dataframe0)
    
    return recap,Kmeans0

# Tests :

Les leviers d'action sont la période de temps où les données sont récupérées et la fréquence de référence. Le but ici est donc de trouver la meilleure combinaison temps / fréquence pour diminuer la consommation optimale totale.

On exécute donc la fonction précédente pour différentes combinaisons fréquence/temps, que l'on stocke dans "recap". Ensuite, une fois que l'on a la consommation énergétique optimale pour chaque cluster, il reste à les sommer, puis on stocke le total dans "consos", avant de comparer et de cherche le couple qui engendre la consommation énergétique la plus faible.

In [7]:
liste_freq = [1200000,1300000,1400000,1500000,1600000,1700000,1900000,2000000,2100000,2200000,2300000,2400000]
liste_temps = [5,6,7,8,9,10,11]

m = len(liste_freq)
n = len(liste_temps)
consos = np.zeros((m,n))

for i in range(m):
    freq = liste_freq[i]
    for j in range(n):
        t = liste_temps[j]
        recap,Kmeans0 = clustering_bandits(freq,t,5) # 5 clusters : cf méthode du coude
        consos[i,j] = recap.sum()[1]

NameError: name 'os' is not defined

Dataframe donnant la consommation énergétique totale optimale pour chaque cluster :

In [ ]:
consommations = pd.DataFrame(consos,index=liste_freq,columns=liste_temps)
consommations

In [ ]:
recap,Kmeans0 = clustering_bandits(1200000,7,5)
recap

#### attribue_cluster :

Une fois le clustering effectué à l'étape précédente, cette fonction attribue un cluster à une nouvelle application qui arrive sur le serveur pour être exécutée. Pour se faire, on compare la distance euclidienne entre la nouvelle application et les centroides des clusters Kmeans.



Entrées :
- Kmeans0 : Clustering effectué à l'étape précédente.
- app : nouvelle application dont le cluster est à attribuer. On prend la valeur médiane de chaque covariable avant de lui attribuer un cluster.
- n : nombre de clusters


Sortie :
- c : numéro du cluster attribué à app.

In [ ]:
def attribue_cluster(Kmeans0,app,n):
    app = app.median()
    dist = np.zeros(n)
    centroids = kmeans0.cluster_centers_
    
    for i in range(n):
        dist[i] = np.linalg.norm(test - centroids[i])
        
    return np.argmin(dist)